In [1]:
#Imports
import os
import pandas as pd
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm
import datetime
from statsmodels.stats.diagnostic import het_white
from statsmodels.stats.diagnostic import het_breuschpagan

s = lambda qa: qa**2
a = lambda qwerty: datetime.datetime.now().year - int(qwerty)

In [2]:
casen = pd.read_csv(os.getcwd() + "/datasets/casen/CASEN 2009 con ingresos orig y ajust.csv", index_col=None, header=0, error_bad_lines=False)
casen.count()


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (52,214,249,253,302,303,304,305,306,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,560,562,564,566,568,570,576) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


SEGMENTO    246924
IDVIV       246924
HOGAR       246924
O           246924
FOLIO       246924
             ...  
Y21I5       246924
Y21E5X      246924
Y21N6X      246924
Y21M6       246924
Y21I6       246924
Length: 604, dtype: int64

In [3]:
casen = casen[
    (casen['O20'].notnull()) &
    (casen['ESC'].notnull()) & 
    (casen['O20'] != " ") &
    (casen['ESC'] != " ") &
    (casen['YTOTAJ'] != " ") &
    (casen['YTOTAJ'].notnull()) &
    (casen['YTOTAJ'] != "")
]
casen['ESC'] = casen['ESC'].apply(lambda f : int(f))
casen['EXPO20'] = casen['O20'].apply(a)
casen['YTOTAJ'] = casen['YTOTAJ'].apply(lambda d: float(d))
casen['EXPO20POW2'] = casen['EXPO20'].apply(s)

casen = casen[
    casen['EXPO20'] > 0
]

ols = sm.OLS(casen['YTOTAJ'], sm.add_constant(casen.loc[:,['ESC', 'EXPO20', 'EXPO20POW2']])).fit()
ols.summary()

#3.A
#Se puede interpretar que en este modelo, se estan explicando las variables Escolaridad, Experiencia Laboral 
#y Experiencia Laboral^2, con el modelo propuesto en el enunciado del problema. Todas estas variables, tienen 
#un P-Value 0, eso quiere decir que todas las variables son explicativas respecto de la variable YTOTAJ

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 YTOTAJ   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     2745.
Date:                Mon, 11 Oct 2021   Prob (F-statistic):               0.00
Time:                        20:31:57   Log-Likelihood:            -1.2642e+06
No. Observations:               86425   AIC:                         2.528e+06
Df Residuals:                   86421   BIC:                         2.528e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.033e+05   9897.528    -30.649      0.000   -3.23e+05   -2.84e+05
ESC         3.517e+04    468.938     75.003      0.000    3.43e+04    3.61e+04
EXPO20      2.172e+04    729.055     29.789      0.000    2.03e+04    2.31e+04
EXPO20POW2  -193.7112     12.040    -16.089      0.000    -217.310    -170.113
==============================================================================
Omnibus:                   154402.289   Durbin-Watson:                   1.682
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        352850202.651
Skew:                          12.764   Prob(JB):                         0.00
Kurtosis:                     314.984   Cond. No.                     4.15e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.15e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [4]:
white = het_white(ols.resid,  ols.model.exog)
dict(zip(['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value'], white))

#3.B
#Se puede apreciar que con el test de White, no se aprecia heterosedasticidad del modelo propuesto, con esto,
#se rechachaza la hipotesis alternativa de este test, que establece que si hay heterocedasticidad

{'LM Statistic': 619.9439644169677,
 'LM-Test p-value': 1.204771316947234e-128,
 'F-Statistic': 78.04475648678621,
 'F-Test p-value': 4.071197146445789e-129}

In [5]:

casen = casen[
    (casen['O27'] != ' ') & (casen['O27'] != '')
]
casen['O27'] = casen['O27'].astype(str).astype(int)

casen['SEXO'] = np.select(((casen['SEXO'] == 1), (casen['SEXO'] == 2)), [1, 0])
casen['O19'] =  casen['O19'].astype(str).astype(int)
casen = casen[
    (((casen['EDAD'] <= 65) & (casen['SEXO'] == 1)) |
    ((casen['EDAD'] <= 60) & (casen['SEXO'] == 0))) &
    (casen['O27'] == 1) & 
    (casen['O19'] == 1)
]

casen = pd.get_dummies(casen, columns = ['REGION'])

xss = casen.loc[:,['ESC', 'EXPO20', 'EXPO20POW2', 'SEXO', 'REGION_13']] #Se deja como base a la RM


ols = sm.OLS(casen['YTOTAJ'], sm.add_constant(xss)).fit()
white_new = het_white(ols.resid,  ols.model.exog)
dict(zip(['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value'], white))

#3.C
#Se puede apreciar que con el test de White, no se aprecia heterosedasticidad del modelo propuesto, agregando las 
#variables dicotomicas de SEXO (hombre = 1, mujer = 0) y REGION (REGION_+X), con esto, se rechachaza la hipotesis 
#alternativa de este test, que establece que si hay heterocedasticidad o no para este modelo propuesto y
#con las variables regresoras originales propuestas en el enunciado. 
#Obs: Tener en cuenta de que el R^2 obtenido al agregar esta variable es de una explicacion de un 19.3% del modelo

{'LM Statistic': 619.9439644169677,
 'LM-Test p-value': 1.204771316947234e-128,
 'F-Statistic': 78.04475648678621,
 'F-Test p-value': 4.071197146445789e-129}

In [6]:
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 YTOTAJ   R-squared:                       0.193
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     1782.
Date:                Mon, 11 Oct 2021   Prob (F-statistic):               0.00
Time:                        20:32:00   Log-Likelihood:            -5.2550e+05
No. Observations:               37242   AIC:                         1.051e+06
Df Residuals:                   37236   BIC:                         1.051e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.258e+05   1.25e+04    -26.092      0.000    -3.5e+05   -3.01e+05
ESC         3.934e+04    462.365     85.079      0.000    3.84e+04    4.02e+04
EXPO20      1.055e+04    964.240     10.944      0.000    8663.063    1.24e+04
EXPO20POW2   -69.7262     18.077     -3.857      0.000    -105.158     -34.294
SEXO        1.119e+05   3609.868     30.997      0.000    1.05e+05    1.19e+05
REGION_13   3.578e+04   3758.666      9.518      0.000    2.84e+04    4.31e+04
==============================================================================
Omnibus:                    44342.828   Durbin-Watson:                   1.517
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8988033.564
Skew:                           6.186   Prob(JB):                         0.00
Kurtosis:                      78.094   Cond. No.                     4.55e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.55e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
#3.D

casen = pd.get_dummies(casen, columns = ['T16PT'])
casen = pd.get_dummies(casen, columns = ['T16MT'])


In [8]:

xss = casen.loc[:,['ESC', 'EXPO20', 'EXPO20POW2', 'SEXO',
 'REGION_13', #Tambien se deja como base la Region Metropolitana para esta pregunta
            
 'T16PT_4',
 'T16PT_5',
 'T16PT_6',
 'T16PT_7',
 'T16PT_8',
 'T16PT_9',
 'T16PT_10',
 'T16PT_11',
              
 'T16MT_4',
 'T16MT_5',
 'T16MT_6',
 'T16MT_7',
 'T16MT_8',
 'T16MT_9',
 'T16MT_10',
 'T16MT_11',]]

sm.OLS(casen['YTOTAJ'], sm.add_constant(xss)).fit().summary()

#Se puede apreciar que el estadistico F, tiene un P-Value 0.000, dando cuenta que el modelo es significativo 
#aun añadiendo las variables del nivel de educacion del padre y madre. Esto da cuenta de el modelo estadistico, 
#asumiendo las variables originales propuestas en el enunciado 3.A
#Obs: tener en cuenta que el R^2 de este modelo es de una explicacion de un 25.2% asumiendo las variables 
#dicotomicas de nivel de estudios del padre y madre

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 YTOTAJ   R-squared:                       0.252
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     597.6
Date:                Mon, 11 Oct 2021   Prob (F-statistic):               0.00
Time:                        20:32:03   Log-Likelihood:            -5.2408e+05
No. Observations:               37242   AIC:                         1.048e+06
Df Residuals:                   37220   BIC:                         1.048e+06
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.619e+05   1.22e+04    -21.477      0.000   -2.86e+05   -2.38e+05
ESC         3.298e+04    481.149     68.540      0.000     3.2e+04    3.39e+04
EXPO20      8135.8370    932.470      8.725      0.000    6308.170    9963.504
EXPO20POW2   -33.2109     17.446     -1.904      0.057     -67.406       0.984
SEXO        1.171e+05   3481.065     33.634      0.000     1.1e+05    1.24e+05
REGION_13   2.513e+04   3634.808      6.914      0.000     1.8e+04    3.23e+04
T16PT_4     7.512e+04   7973.598      9.421      0.000    5.95e+04    9.07e+04
T16PT_5     3.416e+04   1.01e+04      3.383      0.001    1.44e+04    5.39e+04
T16PT_6     1.026e+05   1.43e+04      7.201      0.000    7.47e+04    1.31e+05
T16PT_7      1.07e+05   1.97e+04      5.427      0.000    6.84e+04    1.46e+05
T16PT_8     2.251e+05   2.66e+04      8.471      0.000    1.73e+05    2.77e+05
T16PT_9     2.255e+05   2.92e+04      7.710      0.000    1.68e+05    2.83e+05
T16PT_10    3.855e+05   1.38e+04     27.882      0.000    3.58e+05    4.13e+05
T16PT_11     2.24e+04   7347.910      3.049      0.002    7999.596    3.68e+04
T16MT_4     9.267e+04   8067.886     11.486      0.000    7.69e+04    1.08e+05
T16MT_5     2.275e+04   9292.433      2.449      0.014    4539.917     4.1e+04
T16MT_6     7.209e+04   1.47e+04      4.898      0.000    4.32e+04    1.01e+05
T16MT_7     6.547e+04   2.16e+04      3.029      0.002    2.31e+04    1.08e+05
T16MT_8     1.891e+05   3.19e+04      5.922      0.000    1.26e+05    2.52e+05
T16MT_9     1.035e+05   3.57e+04      2.904      0.004    3.36e+04    1.73e+05
T16MT_10    2.778e+05    1.6e+04     17.375      0.000    2.46e+05    3.09e+05
T16MT_11    1.386e+04   7245.964      1.913      0.056    -340.008    2.81e+04
==============================================================================
Omnibus:                    43110.142   Durbin-Watson:                   1.597
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8322898.478
Skew:                           5.883   Prob(JB):                         0.00
Kurtosis:                      75.285   Cond. No.                     1.42e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.42e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""